In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/FairPrune"

/content/drive/MyDrive/FairPrune


In [ ]:
%cd "/content/drive/MyDrive/UNSW/FairPrune"

/content/drive/.shortcut-targets-by-id/1aPwNWOun2_JEryKUu6DPftgTWhsGWBrZ/BiasGDA&Pruning/bin


In [3]:
!python Base.py --config Fitz17k_configs.yml

CUDA is available: True 

Starting... 

{'train': 12809, 'val': 3203}
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100% 44.7M/44.7M [00:00<00:00, 176MB/s]
Traceback (most recent call last):
  File "/content/drive/MyDrive/FairPrune/Base.py", line 253, in <module>
    main(config)
  File "/